<a href="https://colab.research.google.com/github/Rajindra/movie-kompis/blob/main/notebooks/Filmkompis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!git clone https://github.com/Rajindra/movie-kompis
%cd movie-kompis


Cloning into 'movie-kompis'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 33 (delta 7), reused 25 (delta 4), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 15.45 MiB | 20.33 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/content/movie-kompis


In [9]:
!pip install -r requirements.txt

In [40]:
import pandas as pd
import time

# Load existing ratings
ratings = pd.read_csv('data/kids_ratings.csv')

# Define your favorite movie IDs
your_movie_ids = [202517, 364, 131834, 206959, 6377]

# Define your new userId (max + 1)
new_user_id = ratings['userId'].max() + 1

# Create new rating entries — assign 5.0 to all
your_ratings = pd.DataFrame({
    'userId': [new_user_id] * len(your_movie_ids),
    'movieId': your_movie_ids,
    'rating': [5.0] * len(your_movie_ids),
    'timestamp': [int(time.time())] * len(your_movie_ids)
})

# Append to ratings
ratings = pd.concat([ratings, your_ratings], ignore_index=True)

# Save updated ratings
ratings.to_csv('data/kids_ratings.csv', index=False)
print(f"✅ Added ratings for user {new_user_id} and saved to data/kids_ratings.csv")


✅ Added ratings for user 162542 and saved to data/kids_ratings.csv


In [41]:
# Reload updated ratings and movies
ratings = pd.read_csv('data/kids_ratings.csv')
movies = pd.read_csv('data/kids_movies.csv')

# Get the new userId (you just added)
new_user_id = ratings['userId'].max()

# Filter only your ratings
your_rows = ratings[ratings['userId'] == new_user_id]

# Merge with movie titles
your_movies = your_rows.merge(movies, on='movieId', how='left')

# Print full details
print("🎬 Your Ratings:")
print(your_movies[['userId', 'movieId', 'title', 'rating', 'timestamp']])


🎬 Your Ratings:
   userId  movieId                                        title  rating  \
0  162542   202517  Kung Fu Panda: Secrets of the Scroll (2016)     5.0   
1  162542      364                        Lion King, The (1994)     5.0   
2  162542   131834               The Sound of Music Live (2013)     5.0   
3  162542   206959                             Frozen II (2019)     5.0   
4  162542     6377                          Finding Nemo (2003)     5.0   

    timestamp  
0  1745435373  
1  1745435373  
2  1745435373  
3  1745435373  
4  1745435373  


In [42]:
import pandas as pd

# Load kids movie data
movies = pd.read_csv('data/kids_movies.csv')
ratings = pd.read_csv('data/kids_ratings.csv')

print("🎬 Movies sample:")
print(movies.head())

print("⭐ Ratings sample:")
print(ratings.head())


🎬 Movies sample:
   movieId                title                                       genres
0        1     Toy Story (1995)  Adventure|Animation|Children|Comedy|Fantasy
1        2       Jumanji (1995)                   Adventure|Children|Fantasy
2        8  Tom and Huck (1995)                           Adventure|Children
3       13         Balto (1995)                 Adventure|Animation|Children
4       27  Now and Then (1995)                               Children|Drama
⭐ Ratings sample:
   userId  movieId  rating   timestamp
0       1     2161     3.5  1147868609
1       1     6377     4.0  1147868469
2       1     8360     4.0  1147868682
3       2        1     3.5  1141415820
4       2      364     4.5  1141417077


In [43]:
# Step 1: Map userId and movieId to indices

import torch
from sklearn.model_selection import train_test_split

# Map original IDs to index-based IDs
user_map = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
movie_map = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}
reverse_movie_map = {v: k for k, v in movie_map.items()}

ratings['userId'] = ratings['userId'].map(user_map)
ratings['movieId'] = ratings['movieId'].map(movie_map)

n_users = len(user_map)
n_movies = len(movie_map)

print(f"👤 Unique users: {n_users}, 🎥 Unique movies: {n_movies}")


👤 Unique users: 148392, 🎥 Unique movies: 2866


In [44]:
# Step 2: Split and convert to tensors

train_df, test_df = train_test_split(ratings, test_size=0.2, random_state=42)

train_users = torch.tensor(train_df['userId'].values, dtype=torch.long)
train_movies = torch.tensor(train_df['movieId'].values, dtype=torch.long)
train_ratings = torch.tensor(train_df['rating'].values, dtype=torch.float)

test_users = torch.tensor(test_df['userId'].values, dtype=torch.long)
test_movies = torch.tensor(test_df['movieId'].values, dtype=torch.long)
test_ratings = torch.tensor(test_df['rating'].values, dtype=torch.float)


In [45]:
# Step 3: Prepare the DataLoader

from torch.utils.data import DataLoader, TensorDataset

batch_size = 1024

# Create PyTorch datasets
train_dataset = TensorDataset(train_users, train_movies, train_ratings)
test_dataset = TensorDataset(test_users, test_movies, test_ratings)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [46]:
# Step 4: Define the Matrix Factorization Model

import torch.nn as nn

class MatrixFactorization(nn.Module):
    def __init__(self, n_users, n_movies, n_factors=64):
        super(MatrixFactorization, self).__init__()
        self.user_embedding = nn.Embedding(n_users, n_factors)
        self.movie_embedding = nn.Embedding(n_movies, n_factors)
        self.user_bias = nn.Embedding(n_users, 1)
        self.movie_bias = nn.Embedding(n_movies, 1)
        self.global_bias = nn.Parameter(torch.zeros(1))

    def forward(self, user, movie):
        u_emb = self.user_embedding(user)
        m_emb = self.movie_embedding(movie)
        dot = (u_emb * m_emb).sum(1)
        bias = self.user_bias(user).squeeze() + self.movie_bias(movie).squeeze() + self.global_bias
        return dot + bias

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MatrixFactorization(n_users, n_movies).to(device)

In [47]:
# Step 5: Train the Matrix Factorization Model

import torch.optim as optim

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 10

for epoch in range(n_epochs):
    model.train()
    total_loss = 0

    for users, movies, ratings_batch in train_loader:
        users = users.to(device)
        movies = movies.to(device)
        ratings_batch = ratings_batch.to(device)

        optimizer.zero_grad()
        preds = model(users, movies)
        loss = loss_fn(preds, ratings_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1}/{n_epochs}, Loss: {avg_loss:.4f}")



📘 Epoch 1/10, Loss: 52.4711
📘 Epoch 2/10, Loss: 20.7680
📘 Epoch 3/10, Loss: 9.7546
📘 Epoch 4/10, Loss: 5.4050
📘 Epoch 5/10, Loss: 3.4146
📘 Epoch 6/10, Loss: 2.3722
📘 Epoch 7/10, Loss: 1.7550
📘 Epoch 8/10, Loss: 1.3406
📘 Epoch 9/10, Loss: 1.0457
📘 Epoch 10/10, Loss: 0.8499


In [32]:
# Step 6: Make Movie Recommendations

def recommend_movies(user_id_original, top_n=5):
    # Check if user exists
    if user_id_original not in user_map:
        print("❌ User not found.")
        return []

    # Map user ID
    user_id = user_map[user_id_original]
    user_tensor = torch.tensor([user_id] * n_movies).to(device)
    movie_tensor = torch.arange(n_movies).to(device)

    # Predict
    model.eval()
    with torch.no_grad():
        preds = model(user_tensor, movie_tensor).cpu().numpy().flatten()

    # Top-N predictions
    top_indices = preds.argsort()[::-1][:top_n]
    top_movie_ids = [int(reverse_movie_map[int(i)]) for i in top_indices]

    # Fetch titles safely
    titles = []
    for mid in top_movie_ids:
        mid = int(mid)
        title_row = movies[movies['movieId'] == mid]
        if not title_row.empty:
            title = title_row.iloc[0]['title']
            titles.append(str(title))  # Ensure it's a string

    return titles


In [48]:
import pandas as pd

# Re-load the original CSV to ensure all types are clean
movies = pd.read_csv('data/kids_movies.csv')

original_user_id = 162542  # Replace with valid user ID from your dataset

recommendations = recommend_movies(original_user_id, top_n=5)

print("🎯 Recommended Kids Movies:")
for title in recommendations:
    print(" -", title)


🎯 Recommended Kids Movies:
 - Rascals on the Road (2006)
 - Treasure of the Hidden Planet (1997)
 - Pokémon the Movie: Genesect and the Legend Awakened (2013)
 - Rabbit Punch (1948)
 - Balloon Farm (1999)
